In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
import csv
import threading, queue

with open('/kaggle/input/nbme-score-clinical-patient-notes/train.csv', newline='') as csvfile:
     notesreader = csv.reader(csvfile, delimiter='~', quotechar='|')
     for row in notesreader:
        tokens = nltk.word_tokenize(row[0])
        print(tokens)

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# create tokens for each feature/each test phrase

notes = {}

tokens = {} 
replaceChars= '{}[]\'":;-'
stopWords = {'with','to','off'}
corrections = {'PURCHAESD':'PURCHASED','BLEDDING':'BLEEDING','WIEGHT':'WEIGHT','WIGHT':'WEIGHT','HERAT':'HEART','palpitaion': 'palpitation','PALPIATIONS': 'palpitation','attcak': 'attack', 'MENTRUAL':'MENSTRUAL'}
suffixes = {'s','ed', 'ing','ess','ly','ess'}
# {dad: {0:1,1:4}}

q = queue.Queue()

with open('/kaggle/input/nbme-score-clinical-patient-notes/train.csv', newline='') as csvfile:
     notesreader = csv.reader(csvfile, delimiter=',', quotechar='|')
     for row in notesreader:
         for c in replaceChars:
             row[4] = row[4].replace(c, '')
         for t in row[4].split():
            if t in stopWords or len(t) < 4:
                continue
            if t in corrections:
                t = corrections[t]
            if t.upper() in corrections:
                t = corrections[t.upper()]
            
            t= t.upper()
            features = tokens.get(t)
            if features:            
                feature = features.get(row[3])
                if feature: 
                    features[row[3]]=features[row[3]]+1
                else:
                    features[row[3]]=1
            else:
                tokens[t] = {}
                tokens[t][row[3]]=1

features = {}

for key in tokens:
    for feature in tokens[key]:
        featureDict = features.get(feature)
        if featureDict:
            tokensDict = featureDict.get(key)
            if tokensDict:
                tokensDict[key]=tokensDict[key]+tokens[key][feature]
            else:
                features[feature][key]=tokens[key][feature]
        else:
            features[feature] = {}
            features[feature][key]=tokens[key][feature]  

pn_history = {}
with open('/kaggle/input/nbme-score-clinical-patient-notes/patient_notes.csv', newline='') as csvfile:
    notesreader = csv.reader(csvfile, delimiter=',', quotechar='|')
    
        
# for each token in test match the feature that has the highest tokens 
# 2) Give priority to the features with in the same case.